In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import DenseNet169
# from keras.applications.densenet import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 64
batchsize = 256
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [4]:
K.clear_session()
# base_model = MobileNet(input_shape=(size, size, 3), alpha=1., weights='imagenet', include_top=False)
# x = base_model.output
# # x = GlobalAveragePooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)

# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# x = Dense(64)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)

# predictions = Dense(NCATS, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

model = DenseNet169(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 70, 70, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 32, 64)   3136        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [5]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, 
                               preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [6]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [7]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [8]:
model_prefix = 'densenet169_raw_head_{}_lw{}'.format(size, lw)
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}.model".format(model_prefix),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
]

In [9]:
hists = []
hist = model.fit_generator(  
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/50
1294/1293 [==============================] - 365s 282ms/step - loss: 2.4368 - categorical_crossentropy: 2.4368 - categorical_accuracy: 0.4480 - top_3_accuracy: 0.6468 - val_loss: 2.2058 - val_categorical_crossentropy: 2.2058 - val_categorical_accuracy: 0.4850 - val_top_3_accuracy: 0.6981

Epoch 00001: val_categorical_accuracy improved from -inf to 0.48497, saving model to ./models/densenet169_raw_head_64_lw6.model
Epoch 2/50
1294/1293 [==============================] - 334s 258ms/step - loss: 1.6008 - categorical_crossentropy: 1.6008 - categorical_accuracy: 0.6074 - top_3_accuracy: 0.8000 - val_loss: 2.6196 - val_categorical_crossentropy: 2.6196 - val_categorical_accuracy: 0.4205 - val_top_3_accuracy: 0.6299

Epoch 00002: val_categorical_accuracy did not improve from 0.48497
Epoch 3/50
1294/1293 [==============================] - 335s 259ms/step - loss: 1.4168 - categorical_crossentropy: 1.4168 - categorical_accuracy: 0.6475 - top_3_accuracy: 0.8323 - val_loss: 4.5128 - val_

1294/1293 [==============================] - 338s 261ms/step - loss: 0.9404 - categorical_crossentropy: 0.9404 - categorical_accuracy: 0.7576 - top_3_accuracy: 0.9022 - val_loss: 0.9811 - val_categorical_crossentropy: 0.9811 - val_categorical_accuracy: 0.7476 - val_top_3_accuracy: 0.8973

Epoch 00021: val_categorical_accuracy did not improve from 0.75212
Epoch 22/50
1294/1293 [==============================] - 338s 261ms/step - loss: 0.9366 - categorical_crossentropy: 0.9366 - categorical_accuracy: 0.7594 - top_3_accuracy: 0.9029 - val_loss: 0.9467 - val_categorical_crossentropy: 0.9467 - val_categorical_accuracy: 0.7566 - val_top_3_accuracy: 0.9011

Epoch 00022: val_categorical_accuracy improved from 0.75212 to 0.75659, saving model to ./models/densenet169_raw_head_64_lw6.model
Epoch 23/50
1294/1293 [==============================] - 338s 261ms/step - loss: 0.9369 - categorical_crossentropy: 0.9369 - categorical_accuracy: 0.7584 - top_3_accuracy: 0.9029 - val_loss: 0.9666 - val_catego


Epoch 00040: val_categorical_accuracy improved from 0.79553 to 0.79782, saving model to ./models/densenet169_raw_head_64_lw6.model
Epoch 41/50
1294/1293 [==============================] - 334s 258ms/step - loss: 0.7936 - categorical_crossentropy: 0.7936 - categorical_accuracy: 0.7947 - top_3_accuracy: 0.9209 - val_loss: 0.7880 - val_categorical_crossentropy: 0.7880 - val_categorical_accuracy: 0.7979 - val_top_3_accuracy: 0.9215

Epoch 00041: val_categorical_accuracy improved from 0.79782 to 0.79785, saving model to ./models/densenet169_raw_head_64_lw6.model
Epoch 42/50
1294/1293 [==============================] - 335s 259ms/step - loss: 0.7922 - categorical_crossentropy: 0.7922 - categorical_accuracy: 0.7949 - top_3_accuracy: 0.9215 - val_loss: 0.7824 - val_categorical_crossentropy: 0.7824 - val_categorical_accuracy: 0.8000 - val_top_3_accuracy: 0.9224

Epoch 00042: val_categorical_accuracy improved from 0.79785 to 0.80000, saving model to ./models/densenet169_raw_head_64_lw6.model
Ep

In [10]:
model.load_weights('./models/{}.model'.format(model_prefix))

In [11]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

(112199, 3) (112199, 64, 64, 1)
Test array memory 1.71 GB


In [12]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

112199/112199 [==============================] - 49s 440us/step


(112199, 3)

In [13]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

(112199, 3)

In [14]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle camel
3,9000038588854897,mountain tent church
4,9000052667981386,fireplace campfire pineapple


In [15]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}.csv'.format(model_prefix), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)

save result
upload result
cmd: kaggle competitions submit -c quickdraw-doodle-recognition -f ../result/densenet169_raw_head_64_lw6.csv.7z -m "submit"
